In [1]:
import pyodbc
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import MaxAbsScaler

import warnings
warnings.filterwarnings("ignore")

from pycaret.anomaly import *
from datetime import datetime

In [32]:
def df_facturas(consulta_SQL):
    ## Conexion al DWH
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()
    
    #Cargue de la data desde el ERP de Dispapeles y se guarda en df
    cursor.execute(consulta_SQL)
    rows = cursor.fetchall()
    
    df = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])

    return df

In [3]:
distrito = 10
zona = 69
fecha_inicio = '2022-01-01'
fecha_final = '2023-02-28'

codigo_SQL = f"SELECT A.VTANUMDOC, A.VTAPRDCOD, A.VTANIT, A.VTANITVND, SUM(A.VTAVLRVTA) AS 'Venta', SUM(A.VTAUNDINV) AS 'Unidades', ROUND(SUM(A.VTAVLRVTA) / SUM(A.VTAUNDINV),2) AS 'Precio unitario', ROUND(SUM(A.VTAVLRDTO) / SUM(A.VTAUNDINV),2) AS 'Descuento unitario' FROM V_VTA_VTAHEC A WHERE A.DSTCOD = {distrito} AND A.VTAZNA = {zona} AND A.VTAFCH BETWEEN '{fecha_inicio}' AND '{fecha_final}' GROUP BY A.VTANUMDOC, A.VTAPRDCOD, A.VTANIT, A.VTANITVND HAVING SUM(A.VTAUNDINV) > 0"

In [6]:
df = df_facturas(codigo_SQL)

In [11]:
anomaly_exp = AnomalyExperiment()
anomaly_exp.setup(df, session_id= 42)

In [13]:
iforest = anomaly_exp.create_model('iforest')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:21:39
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
iforest_anomalies = anomaly_exp.assign_model(iforest)

In [17]:
anomaly_exp.plot_model(iforest, plot= 'tsne')

In [33]:
distrito = 10
zona = 69
fecha_inicio = '2023-03-01'
fecha_final = '2023-03-31'

codigo_SQL_2 = f"SELECT A.VTANUMDOC, A.VTAPRDCOD, A.VTANIT, A.VTANITVND, SUM(A.VTAVLRVTA) AS 'Venta', SUM(A.VTAUNDINV) AS 'Unidades', ROUND(SUM(A.VTAVLRVTA) / SUM(A.VTAUNDINV),2) AS 'Precio unitario', ROUND(SUM(A.VTAVLRDTO) / SUM(A.VTAUNDINV),2) AS 'Descuento unitario' FROM V_VTA_VTAHEC A WHERE A.DSTCOD = {distrito} AND A.VTAZNA = {zona} AND A.VTAFCH BETWEEN '{fecha_inicio}' AND '{fecha_final}' GROUP BY A.VTANUMDOC, A.VTAPRDCOD, A.VTANIT, A.VTANITVND HAVING SUM(A.VTAUNDINV) > 0"
df_2 = df_facturas(codigo_SQL_2)

In [37]:
iforest_pred = anomaly_exp.predict_model(iforest, data= df_2)
iforest_pred_1 = iforest_pred[iforest_pred["Anomaly"] == 1]

In [38]:
iforest_pred_1

,VTANUMDOC_1431120,VTANUMDOC_1431308,VTANUMDOC_1431336,VTANUMDOC_1431422,VTANUMDOC_1431430,VTANUMDOC_1431435,VTANUMDOC_1431599,VTANUMDOC_1431643,VTANUMDOC_1431658,VTANUMDOC_1431695,...,Descuento unitario_19719.960000,Descuento unitario_3937.680000,Descuento unitario_5758.630000,Descuento unitario_3178.960000,Descuento unitario_38086.420000,Descuento unitario_14264.310000,Descuento unitario_10971.670000,Descuento unitario_10539.300000,Anomaly,Anomaly_Score
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000023
262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.001023


In [39]:
l = len(iforest_pred_1)

for x in range(l):
    print(iforest_pred_1.columns[x])

VTANUMDOC_1431120
VTANUMDOC_1431308
